In [2]:
# --- Dash + libs ---
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64, os

import pandas as pd

# --- CRUD module ---
from animal_shelter import AnimalShelter

# --- DB connection ---
username = "aacuser"
password = "lebron"
db = AnimalShelter(username, password)

def _to_df(records):
    df = pd.DataFrame.from_records(records)
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    return df

try:
    df = _to_df(db.read({}))
except Exception:
    df = pd.DataFrame()

# Default empty schema so DataTable has columns even if DB is empty
if df.empty:
    df = pd.DataFrame(columns=[
        "age_upon_outcome_in_weeks","animal_type","breed","color","name",
        "sex_upon_outcome","location_lat","location_long"
    ])

# --- JupyterDash app ---
app = JupyterDash(__name__)

# logo (optional)
logo_paths = ["Grazioso Salvare Logo.png","Grazioso_Salvare_Logo.png","logo.png"]
logo_path = next((p for p in logo_paths if os.path.exists(p)), None)
logo_img = base64.b64encode(open(logo_path,"rb").read()).decode() if logo_path else None

header_row = html.Div(
    style={"display":"flex","alignItems":"center","gap":"16px"},
    children=[
        html.A(
            href="https://www.snhu.edu", target="_blank",
            children=html.Img(
                src=f"data:image/png;base64,{logo_img}" if logo_img else None,
                style={"height":"60px"}
            )
        ),
        html.Div([
            html.H1("Grazioso Salvare — Rescue Dog Dashboard", style={"margin":"0"}),
            html.Div("Created by Christopher Prempeh", style={"fontStyle":"italic"})
        ])
    ]
)

filter_ctrl = dcc.RadioItems(
    id="filter-type",
    options=[
        {"label":"Water Rescue","value":"water"},
        {"label":"Mountain/Wilderness Rescue","value":"mountain"},
        {"label":"Disaster / Individual Tracking","value":"disaster"},
        {"label":"Reset","value":"reset"},
    ],
    value="reset",
    labelStyle={"display":"inline-block","marginRight":"14px"}
)

datatable = dash_table.DataTable(
    id="datatable-id",
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict("records"),
    page_size=10,
    sort_action="native",
    filter_action="native",
    row_selectable="single",
    selected_rows=[0] if len(df) else [],
    style_table={"overflowX":"auto"},
    style_cell={"minWidth":"100px","width":"100px","maxWidth":"250px","textAlign":"left"},
    style_header={"fontWeight":"bold"},
)

app.layout = html.Div([
    header_row,
    html.Hr(),
    html.Div(filter_ctrl),
    html.Hr(),
    datatable,
    html.Br(),
    html.Hr(),
    html.Div(className="row", style={"display":"flex","gap":"16px"}, children=[
        html.Div(id="graph-id", className="col s12 m6", style={"flex":"1"}),
        html.Div(id="map-id", className="col s12 m6", style={"flex":"1"})
    ])
])

# --- filter sets from specs ---
WATER_BREEDS = ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]
MOUNTAIN_BREEDS = ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
DISASTER_BREEDS = ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]

def build_query(kind:str):
    if kind=="water":
        return {"$and":[
            {"breed":{"$in": WATER_BREEDS}},
            {"sex_upon_outcome":"Intact Female"},
            {"age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}}
        ]}
    if kind=="mountain":
        return {"$and":[
            {"breed":{"$in": MOUNTAIN_BREEDS}},
            {"sex_upon_outcome":"Intact Male"},
            {"age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}}
        ]}
    if kind=="disaster":
        return {"$and":[
            {"breed":{"$in": DISASTER_BREEDS}},
            {"sex_upon_outcome":"Intact Male"},
            {"age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}}
        ]}
    return {}

# --- callbacks ---
@app.callback(Output("datatable-id","data"), Input("filter-type","value"))
def update_dashboard(filter_type):
    records = db.read(build_query(filter_type))
    return _to_df(records).to_dict("records")

@app.callback(Output("graph-id","children"), Input("datatable-id","derived_virtual_data"))
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData is not None else df
    if dff.empty or "breed" not in dff.columns:
        fig = px.pie(values=[1], names=["No data"], title="Animals by Breed")
    else:
        vc = dff["breed"].value_counts().reset_index()
        vc.columns = ["breed","count"]
        fig = px.pie(vc, names="breed", values="count", title="Animals by Breed")
    return dcc.Graph(figure=fig)

@app.callback(Output("datatable-id","style_data_conditional"), Input("datatable-id","selected_columns"))
def update_styles(selected_columns):
    return [{'if':{'column_id':i},'background_color':'#D2F3FF'} for i in (selected_columns or [])]

@app.callback(Output("map-id","children"),
             [Input("datatable-id","derived_virtual_data"),
              Input("datatable-id","derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    dff = pd.DataFrame(viewData) if viewData is not None else df
    if dff.empty:
        center = [30.27,-97.74]
        return [dl.Map(style={'width':'100%','height':'500px'}, center=center, zoom=10, children=[dl.TileLayer()])]
    row = selected_rows[0] if selected_rows else 0

    # tolerant column lookup
    lat_col  = "location_lat"  if "location_lat"  in dff.columns else (dff.columns[13] if len(dff.columns)>13 else None)
    lon_col  = "location_long" if "location_long" in dff.columns else (dff.columns[14] if len(dff.columns)>14 else None)
    name_col = "name"          if "name"          in dff.columns else (dff.columns[4]  if len(dff.columns)>4  else None)
    breed_col= "breed"         if "breed"         in dff.columns else (dff.columns[2]  if len(dff.columns)>2  else None)

    lat = float(dff.iloc[row][lat_col]) if lat_col and pd.notna(dff.iloc[row][lat_col]) else 30.27
    lon = float(dff.iloc[row][lon_col]) if lon_col and pd.notna(dff.iloc[row][lon_col]) else -97.74
    breed = str(dff.iloc[row][breed_col]) if breed_col else "Unknown"
    name  = str(dff.iloc[row][name_col])  if name_col  else "Unknown"

    return [dl.Map(style={'width':'100%','height':'500px'}, center=[lat,lon], zoom=11, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat,lon], children=[
            dl.Tooltip(breed),
            dl.Popup([html.H4("Animal Name"), html.P(name)])
        ])
    ])]

# --- run inline through Jupyter proxy (fixes connection refused) ---
app.run_server(mode="inline", port=8051, debug=False)

 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_alive_0f3c60ed-0986-4d36-906d-2b3a331600a7 HTTP/1.1" 200 -


127.0.0.1 - - [20/Oct/2025 11:05:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1752168217.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1752168217.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2025 11:05:01] "GET /_dash-component-suites/dash/dash_table/bundle.v5_2_2m1752168217.js HTTP/1.1" 200 -
127.0.0